Calculator

- encoder로 추출한 output 파일과 input 값의 embedding 값들의 코사인 유사도를 계산



In [1]:
pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 마운트
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [2]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
# 데이터를 NumPy array로 변환하는 함수
def parse_and_convert(value):

    value = value.strip('[]')
    return np.array([float(x) for x in value.split(',')])

In [47]:
# output 파일 로드
csv_file_path = 'output_first.csv'  # Replace with the actual path to your CSV file
# csv_file_path = 'output_total.csv'  # Replace with the actual path to your CSV file
df = pd.read_csv(csv_file_path)

# NumPy array로 변환
df['embeddings'] = df['embeddings'].apply(parse_and_convert)

# 타겟값 지정
target_column = 'embeddings'
target_data = df[target_column]

In [42]:
df.shape

(40128, 4)

CASE1. 초록을 새로 입력 시 가장 유사한 초록과 그 ipc코드를 반환


In [8]:
# 평균 풀링 - 주의 마스크(attention_mask)를 고려하여 평균 계산
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# 학습된 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained('jhgan/ko-sroberta-multitask')
model = AutoModel.from_pretrained('jhgan/ko-sroberta-multitask')

# 입력 문장
input_sentence = input('초록을 입력하세요 : ')

# 입력문장 인코딩
encoded_input = tokenizer(input_sentence, padding=True, truncation=True, return_tensors='pt')
with torch.no_grad():
    model_output = model(**encoded_input)
input_embedding = mean_pooling(model_output, encoded_input['attention_mask']).numpy()


# 코사인 유사도 계산
similarity_scores = cosine_similarity(input_embedding, np.stack(target_data.apply(np.array), axis=0))


# 가장 유사한 인덱스 추출
most_similar_row_index = np.argmax(similarity_scores)
result_1 = df.loc[most_similar_row_index, '초록']
result_2 = df.loc[most_similar_row_index, 'ipc코드']

# 결과 도출
print("\n가장 유사한 초록 \n->",result_1)
print("\n유사 ipc코드\n->",result_2)

c:\Users\yeppi\miniconda3\envs\gyubin\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yeppi\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)



가장 유사한 초록 
-> 단면도 및 정면 뷰

유사 ipc코드
-> G10K


CASE2. ipc 코드 맞출 확률 계산

In [59]:
from tqdm import tqdm
# 확률계산 함수 설정
def calculate_matching_ipc_probability(df, tokenizer, model, num):
    matching_count = 0
    big_count = 0
    total_count = 0
    
    A = 0
    B = 0
    C = 0
    D = 0
    E = 0
    F = 0
    G = 0
    
    a = 0
    b = 0
    c = 0
    d = 0
    e = 0
    f = 0
    g = 0
    
    a_same = 0
    b_same = 0
    c_same = 0
    d_same = 0
    e_same = 0
    f_same = 0
    g_same = 0
    
    
    
    for _ in tqdm(range(num)):
        # df 내 100개 데이터 랜덤 추출

        random_index = np.random.randint(len(df)-1)
        random_row = df.iloc[random_index]
        df = df.drop(random_index)
        df = df.reset_index(drop = True)
        
        # 입력 문장
        input_sentence = random_row['초록']

        # 입력 문장 인코딩
        encoded_input = tokenizer(input_sentence, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            model_output = model(**encoded_input)
        input_embedding = mean_pooling(model_output, encoded_input['attention_mask']).numpy()

        # 코사인 유사도 계산
        similarity_scores = cosine_similarity(input_embedding, np.stack(df['embeddings'].to_numpy(), axis=0))

        # 가장 유사한 인덱스 추출
        most_similar_row_index = np.argmax(similarity_scores)
        most_similar_row = df.iloc[most_similar_row_index]

        # ipc코드 매칭 여부
        ran_IPC = random_row['ipc코드']
        sim_IPC = most_similar_row['ipc코드']
        
        if ran_IPC == sim_IPC:
            matching_count += 1
        if ran_IPC[0] == sim_IPC[0]:
            big_count += 1
        
        if ran_IPC[0] == 'A':
            A += 1
            if sim_IPC[0] == 'A':
                a += 1
                if ran_IPC == sim_IPC:
                    a_same += 1
        if ran_IPC[0] == 'B':
            B += 1
            if sim_IPC[0] == 'B':
                b += 1
                if ran_IPC == sim_IPC:
                    b_same += 1                
        if ran_IPC[0] == 'C':
            C += 1
            if sim_IPC[0] == 'C':
                c += 1
                if ran_IPC == sim_IPC:
                    c_same += 1                
        if ran_IPC[0] == 'D':
            D += 1
            if sim_IPC[0] == 'D':
                d += 1
                if ran_IPC == sim_IPC:
                    d_same += 1                
        if ran_IPC[0] == 'E':
            E += 1
            if sim_IPC[0] == 'E':
                e += 1
                if ran_IPC == sim_IPC:
                    e_same += 1                
        if ran_IPC[0] == 'F':
            F += 1
            if sim_IPC[0] == 'F':
                f += 1
                if ran_IPC == sim_IPC:
                    f_same += 1                
        if ran_IPC[0] == 'G':
            G += 1
            if sim_IPC[0] == 'G':
                g += 1
                if ran_IPC == sim_IPC:
                    g_same += 1                
                                                                                                   
        

        total_count += 1
        # 복원
        df = df.append(random_row)
    
    big_probability = big_count / total_count
    matching_probability = matching_count / total_count
    last = [a/A, b/B, c/C, d/D, e/E, f/F, g/G]
    semi = [a_same/a, b_same/b, c_same/c, d_same/d, e_same/e, f_same/f, g_same/g]
    print('*' * 30)
    print(f"ipc코드 4자리 맞출 확률 : {matching_probability*100}'%' ")
    print(f"ipc코드 대분류 맞출 확률 : {big_probability*100}'%' ")
    print(f"소분류 A 맞출 확률 : {last[0]*100}'%' ")
    print(f"소분류 B 맞출 확률 : {last[1]*100}'%' ")
    print(f"소분류 C 맞출 확률 : {last[2]*100}'%' ")
    print(f"소분류 D 맞출 확률 : {last[3]*100}'%' ")
    print(f"소분류 E 맞출 확률 : {last[4]*100}'%' ")
    print(f"소분류 F 맞출 확률 : {last[5]*100}'%' ")
    print(f"소분류 F 맞출 확률 : {last[6]*100}'%' ")
    print('*' * 30)
    print(f"4자리 맞출 확률 : {(a_same + b_same + c_same + d_same + e_same + f_same + g_same)/(a+b+c+d+e+f+g)}")
    print(f"A 4자리 맞출 확률 : {semi[0]*100}'%' ")
    print(f"B 4자리 맞출 확률 : {semi[1]*100}'%' ")
    print(f"C 4자리 맞출 확률 : {semi[2]*100}'%' ")
    print(f"D 4자리 맞출 확률 : {semi[3]*100}'%' ")
    print(f"E 4자리 맞출 확률 : {semi[4]*100}'%' ")
    print(f"F 4자리 맞출 확률 : {semi[5]*100}'%' ")
    print(f"F 4자리 맞출 확률 : {semi[6]*100}'%' ")    
    
    return matching_probability, big_probability, last




In [61]:
import random
import warnings
warnings.filterwarnings('ignore')

seed = 0

random.seed(seed)
np.random.seed(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(seed)
if device == 'cuda':
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# 결과 도출
matching_probability, big_probability, last = calculate_matching_ipc_probability(df.copy(), tokenizer, model, 1000)

 17%|█▋        | 174/1000 [01:11<05:36,  2.45it/s]